In [ ]:
import math
import numpy as np
import pandas as pd
import os
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

from keras.models import Sequential
from keras import layers
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D

In [ ]:
ticker = yf.Ticker("RELIANCE.NS")
data = ticker.history(
    # period = "20y",
    start = '2002-01-01',
    end = '2019-12-31',
    interval = "1d")

data.sort_values('Date', inplace=True, ascending=True)
# data.reset_index(inplace = True)
data = data[data['Volume']>0] #To filter out garbage values
data.drop(['Dividends', 'Stock Splits'], axis = 1, inplace=True)

In [ ]:
# Feature engineering
data['Change'] = 100*((data['Close']-data['Close'].shift(1))/data['Close'].shift(1))
data['Signal'] = 2
data['Signal'] = np.where(data['Change']>= 1, 3, data['Signal'])
data['Signal'] = np.where(data['Change']>= 2.5, 4, data['Signal'])
data['Signal'] = np.where(data['Change']<=-1, 1, data['Signal'])
data['Signal'] = np.where(data['Change']<=-2.5, 0, data['Signal'])
data = data[1:] # Drop first row because NAN
data.head()

In [ ]:
def createImage(df, height_multiplier, min_val, max_val):
    image_width = len(df)
    image_height = height_multiplier*image_width
    image = np.zeros((image_height, image_width))
    factor = image_height/(max_val-min_val)
    for i in range(len(df)):
        if(df.Open.iloc[i]<=df.Close.iloc[i]):
            candle_width = max(int((df.Close.iloc[i] - df.Open.iloc[i])*factor),1)
            start = int((max_val - df.Close.iloc[i])*factor)
            image[start:start+candle_width,i] = 128
        else:
            candle_width = max(int((df.Open.iloc[i] - df.Close.iloc[i])*factor),1)
            start = int((max_val - df.Open.iloc[i])*factor)
            image[start:start+candle_width,i] = 255
    return image

In [ ]:
def dataGenerator(df, timestep, window, height_multiplier=5, batch_size=16):
    c = 0
    while(True):
        image_width = timestep
        image_height = height_multiplier*image_width

        img = np.zeros((batch_size, image_height, image_width, 1)).astype('float')
        y = np.zeros((batch_size))

        for i in range(c, c+batch_size):
            # Create slice of dataframe and Retrieve image
            data = df[window+i:window+i+timestep]
            max_val = df[i:window+i+timestep].High.max()
            min_val = df[i:window+i+timestep].Low.min()
            image = createImage(data, height_multiplier, min_val, max_val)

            # Get prediction
            pred = df.Signal.iloc[window+i+timestep]

            # Add to respective batch sized arrays
            image = image.reshape(image.shape[0], image.shape[1], 1)
            img[i-c] = image
            y[i-c] = pred

        c+=batch_size
        if(c + batch_size+window+timestep >= len(df)):
            c=0
        yield img, y

In [ ]:
model = Sequential([
  layers.Conv2D(8, 3, padding='same', activation='relu', input_shape=(int(timestep*height_multiplier), timestep, 1)),
  layers.MaxPooling2D(),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dropout(0.2),
  layers.Dense(64, activation='relu'),
  layers.Dense(5, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

In [ ]:
batch_size = 64
timestep = 50
height_multiplier = 2
window = int(timestep*1)

split_test = int(len(data)*0.95)
split_val = int(len(data)*0.85)

train_gen = dataGenerator(data[:split_val], timestep, window, height_multiplier, batch_size)
val_gen = dataGenerator(data[split_val:split_test], timestep, window, height_multiplier, batch_size)
test_gen = dataGenerator(data[split_test:], timestep, window, height_multiplier, batch_size)

history = model.fit(x=train_gen,
                    validation_data=val_gen,
                    epochs=15,
                    steps_per_epoch = split_val // batch_size,
                    validation_steps = (split_test-split_val) // batch_size,
                    shuffle=False,
                    verbose=True)

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()

In [ ]:
print(model.evaluate(test_gen, steps = (len(data)-split_test)//batch_size))